## Yo this notebook is lit

In [109]:
import pandas as pd
import numpy as np
#from skimage.io import imshow
from sklearn.model_selection import train_test_split

In [110]:
import keras
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
y_train_labels = [class_labels[int(val)] for val in y_train]
h = 32
w = 32

X_train = X_train/255.0
X_test = X_test/255.0
X_train = X_train - 1/2
X_test = X_test - 1/2

# Takes only the classes we are concerned with 
sub_sample = [1, 4, 9]
X_sub_train = []
y_sub_train = []
for i in range(0,len(y_train)):
    if y_train[i][0] in sub_sample:
        X_sub_train.append(X_train[i])
        if y_train[i][0] == 1 or y_train[i][0] == 9:
            y_sub_train.append([0])
        if y_train[i][0] == 4:
            y_sub_train.append([1])

        
X_sub_test = []
y_sub_test = []
for i in range(0,len(y_test)):
    if y_test[i][0] in sub_sample:
        X_sub_test.append(X_test[i])
        # If it's a truck or automobile
        if y_test[i][0] == 1 or y_test[i][0] == 9:
            y_sub_test.append([0])
        # If it's a deer
        if y_test[i][0] == 4:
            y_sub_test.append([1])

X_sub_train = np.array(X_sub_train)
X_sub_test = np.array(X_sub_test)
y_sub_train = np.array(y_sub_train)
y_sub_test = np.array(y_sub_test)
        
sub_class_labels = ['automobile', 'deer']
y_sub_train_labels = [sub_class_labels[int(val)] for val in y_sub_train]

NUM_CLASSES = len(sub_class_labels)
y_sub_train_ohe = keras.utils.to_categorical(y_sub_train, NUM_CLASSES)
y_sub_test_ohe = keras.utils.to_categorical(y_sub_test, NUM_CLASSES)

In [111]:
#print(X_sub_train)
#print(X_train)

In [112]:
#imshow(X_sub_train[0].squeeze())

In [113]:
print(len(X_sub_train))
print(len(y_sub_train))
print(len(X_sub_test))
print(len(y_sub_test))

15000
15000
3000
3000


In [114]:
#X_c = np.concatenate([X_train,X_test], axis=0)
#y_c = np.concatenate([y_train,y_test], axis=0).flatten()
#vectorized_color = np.reshape(X_c,(len(X_c),32*32,3))
#color_pixels = np.reshape(vectorized_color,(len(vectorized_color)*32*32,3))

In [115]:
#bw_pixels = [(0.21*pixel[0])+(0.72*pixel[1])+(0.07*pixel[2]) for pixel in color_pixels]#convert rgb to b/w
#vectorized_bw = np.reshape(bw_pixels,(len(X),32*32))#reconstruct to list in image vectors
#vectorized_bw.dump("data/vectorized_bw")

In [116]:
'''vectorized_bw_all = np.load("data/vectorized_bw")
print(vectorized_bw_all.shape)

#we might need these DF's later
df_bw = pd.DataFrame(data=vectorized_bw_all)
df_bw = df_bw.assign(y=pd.Series(y,name='y'))
#df_bw_sample = df_bw.sample(5000)
X = df_bw.drop('y', axis=1).as_matrix()
y = df_bw.y.as_matrix()
print(X.shape)
print(y.shape)'''
#now, we can do cv or a train test split

'vectorized_bw_all = np.load("data/vectorized_bw")\nprint(vectorized_bw_all.shape)\n\n#we might need these DF\'s later\ndf_bw = pd.DataFrame(data=vectorized_bw_all)\ndf_bw = df_bw.assign(y=pd.Series(y,name=\'y\'))\n#df_bw_sample = df_bw.sample(5000)\nX = df_bw.drop(\'y\', axis=1).as_matrix()\ny = df_bw.y.as_matrix()\nprint(X.shape)\nprint(y.shape)'

In [117]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [118]:
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average 
from keras.models import Input, Model
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

## CNN #1 

In [179]:
%%time

from keras.layers import average 
from keras.models import Input, Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001

input_holder = Input(shape=(w, h, 3))
conv1 = Conv2D(filters=32,
               input_shape = (w,h,1),
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu')(input_holder)

max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

conv2 = Conv2D(filters=32,
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu')(max1)

max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


# add one layer on flattened output
drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
flat1 = Flatten()(drop1)
dense1 = Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda)
            )(flat1)
drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
dense2_1 = Dense(NUM_CLASSES, 
              activation='sigmoid', 
              kernel_initializer='glorot_uniform',
              kernel_regularizer=l2(l2_lambda)
             )(drop2)

cnn1 = Model(inputs=input_holder,outputs=dense2_1)

# Let's train the model 
cnn1.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn1.fit(X_sub_train, y_sub_train_ohe,
                      #steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Train on 15000 samples, validate on 3000 samples
Epoch 1/10
15000/15000 [==============================] - 5s - loss: 0.2708 - acc: 0.9019 - val_loss: 0.2326 - val_acc: 0.9252
Epoch 2/10
15000/15000 [==============================] - 3s - loss: 0.1602 - acc: 0.9527 - val_loss: 0.1233 - val_acc: 0.9655
Epoch 3/10
15000/15000 [==============================] - 3s - loss: 0.1335 - acc: 0.9624 - val_loss: 0.1352 - val_acc: 0.9585
Epoch 4/10
15000/15000 [==============================] - 3s - loss: 0.1227 - acc: 0.9667 - val_loss: 0.1068 - val_acc: 0.9720
Epoch 5/10
15000/15000 [==============================] - 3s - loss: 0.1151 - acc: 0.9701 - val_loss: 0.1151 - val_acc: 0.9730
Epoch 6/10
15000/15000 [==============================] - 3s - loss: 0.1061 - acc: 0.9739 - val_loss: 0.0981 - val_acc: 0.9780
Epoch 7/10
15000/15000 [==============================] - 3s - loss: 0.1017 - acc: 0.9737 - val_loss: 0.1118 - val_acc: 0.9712
Epoch 8/10
15000/15000 [==============================] - 3s -

In [180]:
yhat_cnn = np.argmax(cnn1.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.95
Accuracy:  0.976666666667


## CNN #1 with Data Expansion

In [121]:
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations.
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts.
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)

In [181]:
cnn1_de = Model(inputs=input_holder,outputs=dense2_1)

# Let's train the model 
cnn1_de.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

datagen.fit(X_sub_train)
cnn1_de.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/10
117/117 [==============================] - 6s - loss: 0.1192 - acc: 0.9680 - val_loss: 0.0873 - val_acc: 0.9795
Epoch 2/10
117/117 [==============================] - 4s - loss: 0.1103 - acc: 0.9706 - val_loss: 0.0859 - val_acc: 0.9797
Epoch 3/10
117/117 [==============================] - 4s - loss: 0.1040 - acc: 0.9732 - val_loss: 0.0812 - val_acc: 0.9818
Epoch 4/10
117/117 [==============================] - 4s - loss: 0.1055 - acc: 0.9738 - val_loss: 0.0860 - val_acc: 0.9787
Epoch 5/10
117/117 [==============================] - 4s - loss: 0.1045 - acc: 0.9719 - val_loss: 0.0936 - val_acc: 0.9742
Epoch 6/10
117/117 [==============================] - 4s - loss: 0.1006 - acc: 0.9738 - val_loss: 0.0818 - val_acc: 0.9812
Epoch 7/10
117/117 [==============================] - 4s - loss: 0.0991 - acc: 0.9737 - val_loss: 0.0805 - val_acc: 0.9828
Epoch 8/10
117/117 [==============================] - 4s - loss: 0.0980 - acc: 0.9755 - val_loss: 0.0904 - val_acc: 0.9803
Epoch 9/10
117/1

In [182]:
yhat_cnn = np.argmax(cnn1_de.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.955
Accuracy:  0.978333333333


## CNN #2: LeNet5

In [176]:
from keras.layers.normalization import BatchNormalization

l2_lambda = 0.0001
input_holder = Input(shape=(w, h, 3))

conv1 = Conv2D(filters=6,kernel_size=(5,5),
               input_shape = (img_wh,img_wh,1), 
               padding='valid', 
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda))(input_holder)
max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
batch1 = BatchNormalization()(max1)
activ1 = Activation("sigmoid")(batch1)

conv2 = Conv2D(filters=16,kernel_size=(5,5), 
               padding='valid', 
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda))(activ1)
max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)
batch2 = BatchNormalization()(max2)
activ2 = Activation("sigmoid")(batch2)

conv3 = Conv2D(filters=120,kernel_size=(1,1), 
               padding='valid', 
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda))(activ2)


# add one layer on flattened output
#drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
flat1 = Flatten()(conv3)
dense1 = Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda)
            )(flat1)
drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
dense2_2 = Dense(NUM_CLASSES, 
              activation='sigmoid', 
              kernel_initializer='glorot_uniform',
              kernel_regularizer=l2(l2_lambda)
             )(drop2)

In [177]:
cnn2 = Model(inputs=input_holder,outputs=dense2_2)

# Let's train the model 
cnn2.compile(loss=w_categorical_crossentropy, # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn2.fit(X_sub_train, y_sub_train_ohe,
                      #steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Train on 15000 samples, validate on 3000 samples
Epoch 1/10
15000/15000 [==============================] - 8s - loss: -5.3369 - acc: 0.6665 - val_loss: -5.3590 - val_acc: 0.6667
Epoch 2/10
15000/15000 [==============================] - 4s - loss: -5.3616 - acc: 0.6667 - val_loss: -5.3650 - val_acc: 0.6667
Epoch 3/10
15000/15000 [==============================] - 4s - loss: -5.3651 - acc: 0.6667 - val_loss: -5.3681 - val_acc: 0.6667
Epoch 4/10
15000/15000 [==============================] - 4s - loss: -5.3676 - acc: 0.6667 - val_loss: -5.3696 - val_acc: 0.6667
Epoch 5/10
15000/15000 [==============================] - 4s - loss: -5.3681 - acc: 0.6667 - val_loss: -5.3698 - val_acc: 0.6667
Epoch 6/10
15000/15000 [==============================] - 4s - loss: -5.3690 - acc: 0.6667 - val_loss: -5.3706 - val_acc: 0.6667
Epoch 7/10
15000/15000 [==============================] - 4s - loss: -5.3697 - acc: 0.6667 - val_loss: -5.3707 - val_acc: 0.6667
Epoch 8/10
15000/15000 [========================

In [126]:
yhat_cnn = np.argmax(cnn2.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.904
Accuracy:  0.956333333333


## CNN #2: LeNet5 with Data Expansion

In [127]:
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations.
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts.
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)

In [135]:
cnn2_de = Model(inputs=input_holder,outputs=dense2_2)

# Let's train the model 
cnn2_de.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

datagen.fit(X_sub_train)
cnn2_de.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/10
117/117 [==============================] - 6s - loss: 0.1526 - acc: 0.9608 - val_loss: 0.1251 - val_acc: 0.9725
Epoch 2/10
117/117 [==============================] - 4s - loss: 0.1460 - acc: 0.9649 - val_loss: 0.1158 - val_acc: 0.9737
Epoch 3/10
117/117 [==============================] - 4s - loss: 0.1511 - acc: 0.9611 - val_loss: 0.1413 - val_acc: 0.9620
Epoch 4/10
117/117 [==============================] - 4s - loss: 0.1474 - acc: 0.9629 - val_loss: 0.1172 - val_acc: 0.9743
Epoch 5/10
117/117 [==============================] - 4s - loss: 0.1462 - acc: 0.9609 - val_loss: 0.1282 - val_acc: 0.9690
Epoch 6/10
117/117 [==============================] - 4s - loss: 0.1440 - acc: 0.9639 - val_loss: 0.1243 - val_acc: 0.9713
Epoch 7/10
117/117 [==============================] - 4s - loss: 0.1420 - acc: 0.9631 - val_loss: 0.1136 - val_acc: 0.9753
Epoch 8/10
117/117 [==============================] - 4s - loss: 0.1411 - acc: 0.9629 - val_loss: 0.3713 - val_acc: 0.8947
Epoch 9/10
117/1

In [137]:
yhat_cnn = np.argmax(cnn2_de.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.717
Accuracy:  0.905333333333


## CNN #1 Ensemble

In [166]:
%%time

from keras.layers import average 
from keras.models import Input, Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001

num_ensembles = 5

input_holder = Input(shape=(w, h, 3))

branches = []
for _ in range(num_ensembles):

    conv1 = Conv2D(filters=32,
                   input_shape = (w,h,1),
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(input_holder)
    
    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

    conv2 = Conv2D(filters=32,
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(max1)
    
    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


    # add one layer on flattened output
    drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(drop1)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2 = Dense(NUM_CLASSES, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    
    # now add this branch onto the master list
    branches.append(dense2)

# that's it, we just need to average the results
ave1 = average(branches)

# here is the secret sauce for setting the network using the 
#   Model API:
cnn_ens1 = Model(inputs=input_holder,outputs=ave1)

# Let's train the model 
cnn_ens1.compile(loss=w_categorical_crossentropy, # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn_ens1.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)])

Epoch 1/10
117/117 [==============================] - 7s - loss: -5.1721 - acc: 0.6655 - val_loss: -5.3147 - val_acc: 0.6667
Epoch 2/10
117/117 [==============================] - 4s - loss: -5.3452 - acc: 0.6656 - val_loss: -5.3393 - val_acc: 0.6667
Epoch 3/10
117/117 [==============================] - 4s - loss: -5.3355 - acc: 0.6672 - val_loss: -5.3507 - val_acc: 0.6667
Epoch 4/10
117/117 [==============================] - 4s - loss: -5.3299 - acc: 0.6680 - val_loss: -5.3568 - val_acc: 0.6667
Epoch 5/10
117/117 [==============================] - 4s - loss: -5.3210 - acc: 0.6689 - val_loss: -5.3598 - val_acc: 0.6667
Epoch 6/10
117/117 [==============================] - 4s - loss: -5.3884 - acc: 0.6648 - val_loss: -5.3617 - val_acc: 0.6667
Epoch 7/10
117/117 [==============================] - 4s - loss: -5.3323 - acc: 0.6684 - val_loss: -5.3634 - val_acc: 0.6667
Epoch 8/10
117/117 [==============================] - 4s - loss: -5.3543 - acc: 0.6672 - val_loss: -5.3650 - val_acc: 0.6667


In [167]:
yhat_cnn = np.argmax(cnn_ens1.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.0
Accuracy:  0.666666666667


In [142]:
yhat_cnn = np.argmax(cnn1.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.991
Accuracy:  0.973


# OKAY BOIS WE JUST TRYNA GET THAT CROSS VALIDATION WORKIN

In [ ]:
from sklearn.model_selection import KFold

l2_lambda = 0.0001





num_folds = 10

model_list1 = []
yhat_list1 = []
y_test_list1 = []

skf = KFold(n_splits=num_folds).split(X_sub_train, y_sub_train_ohe)
for k, (train_indices, test_indices) in enumerate(skf):
    cv_X_train = np.array(X_sub_train[train_indices])
    cv_y_train = np.array(y_sub_train[train_indices])
    cv_y_train_ohe = np.array(y_sub_train_ohe[train_indices])
    
    cv_X_test = np.array(X_sub_train[test_indices])
    cv_y_test = np.array(y_sub_train[test_indices])
    cv_y_test_ohe = np.array(y_sub_train_ohe[test_indices])
    
    
    input_holder = Input(shape=(w, h, 3))
    conv1 = Conv2D(filters=32,
                   input_shape = (w,h,1),
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(input_holder)

    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

    conv2 = Conv2D(filters=32,
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(max1)

    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


    # add one layer on flattened output
    drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(drop1)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2_1 = Dense(2, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    cnn1_de = Model(inputs=input_holder,outputs=dense2_1)

    # Let's train the model 
    cnn1_de.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                    optimizer='rmsprop', # 'adadelta' 'rmsprop'
                    metrics=['accuracy'])
    datagen.fit(cv_X_train)
    cnn1_de.fit_generator(datagen.flow(cv_X_train, cv_y_train_ohe, batch_size=128), 
                          steps_per_epoch=int(len(cv_X_train)/128), # how many generators to go through per epoch
                          epochs=10, verbose=0,
                          validation_data=(cv_X_test, cv_y_test_ohe)
                          #callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                         )
    
    yhat = np.argmax(cnn1_de.predict(cv_X_test), axis=1)
    rec_cnn = mt.recall_score(cv_y_test,yhat)
    acc_cnn = mt.accuracy_score(cv_y_test,yhat)
    print("Recall: ", rec_cnn)
    print("Accuracy: ", acc_cnn)
    print("-----------------------")
    model_list1.append(cnn1_de)
    yhat_list1.append(yhat)
    y_test_list1.append(cv_y_test)

Recall:  0.960151802657
Accuracy:  0.966666666667
-----------------------
Recall:  0.977505112474
Accuracy:  0.979333333333
-----------------------


## "Luke's" loss function

In [165]:
from keras import backend as K
import functools
from itertools import product
import tensorflow as tf
weights=np.array([[1,0],[-1,1]])
def w_categorical_crossentropy(y_true, y_pred):
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = tf.expand_dims(y_pred_max, 1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):

        final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    return K.categorical_crossentropy(y_pred, y_true) * final_mask

In [146]:
%%time

from keras.layers import average 
from keras.models import Input, Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001

num_ensembles = 5

input_holder = Input(shape=(w, h, 3))

branches = []
for _ in range(num_ensembles):

    conv1 = Conv2D(filters=32,
                   input_shape = (w,h,1),
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(input_holder)
    
    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

    conv2 = Conv2D(filters=32,
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(max1)
    
    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


    # add one layer on flattened output
    drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(drop1)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2 = Dense(NUM_CLASSES, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    
    # now add this branch onto the master list
    branches.append(dense2)

# that's it, we just need to average the results
ave = average(branches)

# here is the secret sauce for setting the network using the 
#   Model API:
cnn_ens2 = Model(inputs=input_holder,outputs=ave)

# Let's train the model 
cnn_ens2.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn_ens2.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/10
117/117 [==============================] - 7s - loss: 0.4956 - acc: 0.8506 - val_loss: 0.3133 - val_acc: 0.9228
Epoch 2/10
117/117 [==============================] - 4s - loss: 0.3347 - acc: 0.9074 - val_loss: 0.2687 - val_acc: 0.9452
Epoch 3/10
117/117 [==============================] - 4s - loss: 0.2796 - acc: 0.9316 - val_loss: 0.2065 - val_acc: 0.9652
Epoch 4/10
117/117 [==============================] - 4s - loss: 0.2488 - acc: 0.9416 - val_loss: 0.1971 - val_acc: 0.9625
Epoch 5/10
117/117 [==============================] - 4s - loss: 0.2271 - acc: 0.9504 - val_loss: 0.1820 - val_acc: 0.9670
Epoch 6/10
117/117 [==============================] - 4s - loss: 0.2182 - acc: 0.9506 - val_loss: 0.1932 - val_acc: 0.9613
Epoch 7/10
117/117 [==============================] - 4s - loss: 0.2043 - acc: 0.9564 - val_loss: 0.1912 - val_acc: 0.9595
Epoch 8/10
117/117 [==============================] - 4s - loss: 0.1960 - acc: 0.9601 - val_loss: 0.1597 - val_acc: 0.9707
Epoch 9/10
117/1

In [148]:
from sklearn import metrics as mt
yhat_cnn = np.argmax(cnn_ens.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.945
Accuracy:  0.973


In [147]:
yhat_cnn = np.argmax(cnn_ens2.predict(X_sub_test), axis=1)
rec_cnn = mt.recall_score(y_sub_test,yhat_cnn)
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.896
Accuracy:  0.963


In [144]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#SVG(model_to_dot(cnn2_de).create(prog='dot', format='svg'))

In [83]:
from keras.layers.normalization import BatchNormalization
l2_lambda = 0.0001

num_ensembles = 10
input_holder = Input(shape=(w, h, 3))
branches_le5 = []
for _ in range(num_ensembles):
    conv1 = Conv2D(filters=6,kernel_size=(5,5),
                   input_shape = (img_wh,img_wh,1), 
                   padding='valid', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda))(input_holder)
    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
    batch1 = BatchNormalization()(max1)
    activ1 = Activation("sigmoid")(batch1)

    conv2 = Conv2D(filters=16,kernel_size=(5,5), 
                   padding='valid', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda))(activ1)
    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)
    batch2 = BatchNormalization()(max2)
    activ2 = Activation("sigmoid")(batch2)

    conv3 = Conv2D(filters=120,kernel_size=(1,1), 
                   padding='valid', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda))(activ2)


    # add one layer on flattened output
    #drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(conv3)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2 = Dense(NUM_CLASSES, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    branches_le5.append(dense2)

# that's it, we just need to average the results
ave_le5 = average(branches_le5)   


cnn_le5 = Model(inputs=input_holder,outputs=ave_le5)

# Let's train the model 
cnn_le5.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn_le5.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=10, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/10
117/117 [==============================] - 12s - loss: 0.7890 - acc: 0.8268 - val_loss: 3.2607 - val_acc: 0.6667
Epoch 2/10
117/117 [==============================] - 6s - loss: 0.5357 - acc: 0.8921 - val_loss: 0.7826 - val_acc: 0.6770
Epoch 3/10
117/117 [==============================] - 6s - loss: 0.4470 - acc: 0.9091 - val_loss: 0.6236 - val_acc: 0.8488
Epoch 4/10
117/117 [==============================] - 6s - loss: 0.3921 - acc: 0.9215 - val_loss: 0.5495 - val_acc: 0.9023
Epoch 5/10
117/117 [==============================] - 6s - loss: 0.3479 - acc: 0.9292 - val_loss: 0.3835 - val_acc: 0.9307
Epoch 6/10
117/117 [==============================] - 6s - loss: 0.3261 - acc: 0.9345 - val_loss: 0.3982 - val_acc: 0.93550.
Epoch 7/10
117/117 [==============================] - 6s - loss: 0.3108 - acc: 0.9324 - val_loss: 0.4217 - val_acc: 0.9033
Epoch 8/10
117/117 [==============================] - 6s - loss: 0.2887 - acc: 0.9386 - val_loss: 0.3146 - val_acc: 0.9483
Epoch 9/10
11

In [59]:
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001
img_wh = 32

# Use Kaiming He to regularize ReLU layers: https://arxiv.org/pdf/1502.01852.pdf
# Use Glorot/Bengio for linear/sigmoid/softmax: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf 
cnn = Sequential()

cnn.add(Conv2D(filters=32,
               input_shape = (img_wh,img_wh,1),
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu')) # more compact syntax
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn.add(Conv2D(filters=32,
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
    

# add one layer on flattened output
cnn.add(Dropout(0.25)) # add some dropout for regularization after conv layers
cnn.add(Flatten())
cnn.add(Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda)
       ))
cnn.add(Dropout(0.5)) # add some dropout for regularization, again!
cnn.add(Dense(NUM_CLASSES, 
              activation='softmax', 
              kernel_initializer='glorot_uniform',
              kernel_regularizer=l2(l2_lambda)
             ))

# Let's train the model 
cnn.compile(loss='categorical_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
              optimizer='rmsprop', # 'adadelta' 'rmsprop'
              metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn.fit_generator(datagen.flow(X_subtrain, y_train_ohe, batch_size=128), 
                  steps_per_epoch=int(len(X_train)/128), # how many generators to go through per epoch
                  epochs=50, verbose=1,
                  validation_data=(X_sub_test,y_test_ohe),
                  callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
                 )

ValueError: Error when checking input: expected conv2d_37_input to have shape (None, 32, 32, 1) but got array with shape (2954, 32, 32, 3)